In [1]:
import pymongo
import json
import pandas as pd
import time
import re
import numpy as np
from gmplot import gmplot

In [2]:
client=pymongo.MongoClient()

In [3]:
client.list_database_names()

['admin', 'config', 'local', 'pollution', 'twitterdb']

In [4]:
sample=client.pollution.tweetcollection
population=client.twitterdb.twitter_search

In [5]:
beijingair=re.compile("beijingair",re.IGNORECASE)
beijingair.search(" asdf @Beijingair")

<_sre.SRE_Match object; span=(7, 17), match='Beijingair'>

In [7]:
time.time()

1530552267.236479

In [6]:
oldtime=time.time()
temp=list(population.find({"user":{"id_str":"15527964"}}))
print(time.time()-oldtime)

In [8]:
temp[0]

IndexError: list index out of range

In [9]:
oldtime=time.time()
temp2=list(population.find({"user":{"id":15527964}}))
print(time.time()-oldtime)

In [10]:
temp2[0]

IndexError: list index out of range

In [46]:
# temp1=list(population.find({"text":{"$regex":beijingair}}))

In [61]:
# temp1[0]

In [6]:
# db.tweetcollection.find({"place":{"$ne": None}}).count() # with place tweets
# db.tweetcollection.find({"geo":{"$ne": None}}).count() # with exact location

In [11]:
test=re.compile(r'(^(?=.*air)(?=.*pollution))')
test.findall('air asddf asdf  pollution')

['']

In [7]:
category_0=["r'(^(?=.*air)(?=.*pollution))","空气污染","#airpollution","वायु प्रदुषण","la pollution de l'air","la #pollution","air pollution","l'air #pollution"]

In [8]:
category_0

["r'(^(?=.*air)(?=.*pollution))",
 '空气污染',
 '#airpollution',
 'वायु प्रदुषण',
 "la pollution de l'air",
 'la #pollution',
 'air pollution',
 "l'air #pollution"]

In [9]:
regx = re.compile(r'('+'|'.join(category_0)+')', re.IGNORECASE)
regx

re.compile(r"(r'(^(?=.*air)(?=.*pollution))|空气污染|#airpollution|वायु प्रदुषण|la pollution de l'air|la #pollution|air pollution|l'air #pollution)",
re.IGNORECASE|re.UNICODE)

In [ ]:
# withlocation=list(db.tweetcollection.find({"geo":{"$ne": None},"text":{"$regex":pollution_pattern}}))
# len(withlocation)

In [10]:
# # withlocation[1]
# sample=list(db.tweetcollection.find({'_id':994759019909726208}))
# sample

In [11]:
withplace=list(db.tweetcollection.find({"place":{"$ne": None}}))

In [12]:
len(withplace)

24995

In [13]:
def get_coordinate(this_tweet):
    '''input  dict  tweet
       output list [LONGITUDE LATITUDE]'''
    # There are geo-tagged 3 place store geoinformation 
    # User self-report place 
    # geo  1 Latitude 2 Longitude 
    # coordinates 1 LONGITUDE,2 Latitude 
    # coordinate box same as coordinates
    coordiantes=None
    try:
        coordinates=this_tweet['coordinates']['coordinates']
#         print('coordinates {}'.format(this_tweet['coordinates']['coordinates']))
    except Exception as e:
        pass
    
    try:
        # take average over coordinates box
        coordinates=list(np.array(this_tweet['place']['bounding_box']['coordinates'][0]).mean(axis=0))
    except Exception as e:
        pass

    return coordinates

In [14]:
def get_retweetid(tweet):
    try:
        return(tweet["retweeted_status"]["id"])
    except: 
        return None
    
def get_retweet_username(tweet):
    try:
        return(tweet["retweeted_status"]["user"]["name"])
    except: 
        return None
def get_retweet_followers_count(tweet):
    try:
        return(tweet["retweeted_status"]["user"]["followers_count"])
    except: 
        return None
def get_retweet_fa(tweet):
    try:
        return(tweet["retweeted_status"]["favorite_count"])
    except: 
        return None
def get_retweet_retw(tweet):
    try:
        return(tweet["retweeted_status"]["retweet_count"])
    except: 
        return None
def get_retweet_rep(tweet):
    try:
        return(tweet["retweeted_status"]["reply_count"])
    except: 
        return None
def get_full_text(tweet):
    try:
        return(tweet["extended_tweet"]["full_text"])
    except:
        return(tweet["text"])
    
            
    
def jsontopandas(tweets_data):
    tweets=pd.DataFrame()
    tweets['id'] = list(map(lambda tweet: tweet['_id'], tweets_data))
    tweets['longitude']=list(map(lambda tweet:get_coordinate(tweet)[0], tweets_data))
    tweets['latitude']=list(map(lambda tweet: get_coordinate(tweet)[1], tweets_data))
    tweets['place_country']=list(map(lambda tweet: tweet['place']['country'] if tweet['place']!= None else None, tweets_data))
    tweets['place_full_name']=list(map(lambda tweet: tweet['place']['full_name'] if tweet['place']!= None else None, tweets_data))
    tweets['text']=list(map(lambda tweet: get_full_text(tweet), tweets_data))
    tweets["user_id"]=list(map(lambda tweet:tweet["user"]["id"],tweets_data))
    tweets["user_name"]=list(map(lambda tweet:tweet["user"]["name"],tweets_data))
    tweets["user_followers"]=list(map(lambda tweet:tweet["user"]["followers_count"],tweets_data))
    tweets['user_loc']=list(map(lambda tweet: tweet['user']['location'], tweets_data))
    tweets['lang']=list(map(lambda tweet: tweet['lang'], tweets_data))
    tweets['created_at']=list(map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), tweets_data))
    return tweets

In [15]:
thistweet=jsontopandas(withplace)

In [16]:
# thistweet.head()

In [17]:
thistweet[thistweet.id==994759019909726208]

,id,longitude,latitude,place_country,place_full_name,text,user_id,user_name,user_followers,user_loc,lang,created_at
51,994759019909726208,80.824213,27.014009,India,"Malihabad, India",@lucknowpolice REMINDER III:Courtesy no action...,46348111,Pradeep K Srivastava,1725,India,en,2018-05-11 02:00:06


In [18]:
thistweet.text[51]

'@lucknowpolice REMINDER III:Courtesy no action, heavy trucks/bumpers are freely loving from Bithauli to Kursi road &amp; vice versa via Bhawani bazar &amp; Atal Chauraha.These fast moving trucks are severely damaging newly made road and causing air and noise pollution in residential area https://t.co/sgJ1WpuiZQ'

In [19]:
thistweet.to_csv("../output/tweet_with_location.csv",encoding='utf-8')

In [20]:
load_tweet=pd.read_csv("../output/tweet_with_location.csv")

In [21]:
load_tweet.text[51]

'@lucknowpolice REMINDER III:Courtesy no action, heavy trucks/bumpers are freely loving from Bithauli to Kursi road &amp; vice versa via Bhawani bazar &amp; Atal Chauraha.These fast moving trucks are severely damaging newly made road and causing air and noise pollution in residential area https://t.co/sgJ1WpuiZQ'